In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
os.getcwd()

'/Users/lunal.brown/Personal/DataScience/github/decision_tree_easy'

In [4]:

def get_nodes(df_p, df_grid):
    df_agg=pd.DataFrame([])
    for jlist in df_grid.values:
        jfeat=jlist[0] # 'hour'
        feat_threshold=jlist[1] # 8
        #
        cost_p=df_p[target_col].std(ddof=0) # parentノードのコスト
        n_sample_p=df_p.shape[0]
        #
        df_left=df_p[df_p[jfeat]<feat_threshold].reset_index(drop=True)
        cost_l=df_left[target_col].std(ddof=0)
        n_sample_l=df_left.shape[0]
        #
        df_right=df_p[df_p[jfeat]>=feat_threshold].reset_index(drop=True)
        cost_r=df_right[target_col].std(ddof=0)
        n_sample_r=df_right.shape[0]
        #
        information_gain=cost_p - (n_sample_l * cost_l / n_sample_p) - (n_sample_r * cost_r / n_sample_p)
        # print('feat={}, {:.2f}, {:.3f}, {:.3f}, {:.3f}, {:.4f}'.format( jfeat, feat_threshold, cost_p, cost_l, cost_r, information_gain) )
        df_j=pd.DataFrame({'feature':[jfeat], 'threshold':[feat_threshold], 'information_gain':[information_gain], 'df_left':[df_left], 'df_right':[df_right]})
        df_agg=pd.concat([df_agg, df_j],axis=0)
        del(df_left)
        del(df_right)
    #
    df_agg=df_agg.sort_values(by=['information_gain'],ascending=[False]*1).reset_index(drop=True)
    return df_agg.values.tolist()[0]

In [5]:
df_w=pd.DataFrame([])
df_w=pd.read_csv('weather_data_HanedaJan23rd2022.csv', header=0, encoding='cp932')
df_w.columns=np.arange(0, df_w.shape[1], 1)
df_w=df_w[[0,1,2,3,4,5,6]]
df_w=df_w[5:].reset_index(drop=True)
df_w=df_w.rename(columns={0:'datetime', 1:'temperature', 2:'high', 3:'low', 4:'wind', 5:'quolity', 6:'direction'})
df_w['datetime']=pd.to_datetime(df_w['datetime'], format='%Y/%m/%d %H:%M')
df_w['hour']=df_w['datetime'].dt.hour

list_feature_cols=['hour', 'wind']
target_col='temperature'
for jcol in list_feature_cols + [target_col]:
    df_w[jcol]=df_w[jcol].fillna(value=0).astype(np.float64)
df_w[:5]

,datetime,temperature,high,low,wind,quolity,direction,hour
0,2022-01-14 01:00:00,3.9,8,1,7.2,8,北西,1.0
1,2022-01-14 02:00:00,2.4,8,1,5.5,8,北西,2.0
2,2022-01-14 03:00:00,2.2,8,1,7.5,8,北西,3.0
3,2022-01-14 04:00:00,2.6,8,1,6.8,8,北北西,4.0
4,2022-01-14 05:00:00,2.8,8,1,8.7,8,北北西,5.0


In [11]:
def search_thresholds(df_p, list_feature_cols, n_quantiles=6):
    df_grid=pd.DataFrame([])
    df_quantile=df_p[list_feature_cols].quantile(q=np.linspace(0, 1, n_quantiles).tolist()[1:-1] ).reset_index().reset_index(drop=True).rename(columns={'index':'quantile'})
    print('quantile=\n', df_quantile)
    for jcol in list_feature_cols:
        df_j=df_quantile[[jcol]].reset_index(drop=True)
        df_j['feature']=jcol
        df_j=df_j.rename(columns={jcol:'value'} )
        df_grid=pd.concat([df_grid, df_j],axis=0)
        del(df_j)
    df_grid=df_grid.reset_index(drop=True)
    df_grid=df_grid[['feature', 'value']]
    return df_grid

In [12]:
# ノードを追加していく
node_level_max=3

df_w0=df_w.copy()
n_quantiles=5

df_summary=pd.DataFrame([])
for node_level in np.arange(0,node_level_max,1):
    print('node_level=', node_level)
    if node_level==0:
        df_grid=search_thresholds(df_p=df_w0.copy(), list_feature_cols=list_feature_cols, n_quantiles=n_quantiles)
        jfeat, feat_threshold, information_gain, df_left, df_right=get_nodes(df_p=df_w0.copy(), df_grid=df_grid.copy() )
        df_node_agg=pd.DataFrame({'node_id':[ str(node_level) + '_0' ], 'node_level':[node_level], 'feature':[jfeat], 'threshold':[feat_threshold], 
                              'information_gain':[information_gain], 'df_left':[df_left], 'df_right':[df_right], 'parent_node_id':[np.nan] } )
        del(jfeat)
        del(feat_threshold)
        del(information_gain)
        del(df_left)
        del(df_right)
    elif node_level>=1:
        df_node_parent=df_summary[df_summary['node_level']==node_level-1].reset_index(drop=True)
        df_node_agg=pd.DataFrame([])
        for jdx, jrow in enumerate(df_node_parent.itertuples()):
            print('**')
            df_left_parent=jrow.df_left
            print('left parent type=', type(df_left_parent))
            df_grid=search_thresholds(df_p=df_left_parent.copy(), list_feature_cols=list_feature_cols, n_quantiles=n_quantiles)
            jfeat, feat_threshold, information_gain, df_left, df_right=get_nodes(df_p=df_left_parent.copy(), df_grid=df_grid.copy() )
            df_jl=pd.DataFrame({'node_id':[ str(node_level) + '_' + str(jdx) + 'l' ], 'node_level':[node_level], 'feature':[jfeat], 'threshold':[feat_threshold], 
                              'information_gain':[information_gain], 'df_left':[df_left], 'df_right':[df_right], 'parent_node_id':[jrow.node_id]})
            df_node_agg=pd.concat([df_node_agg, df_jl],axis=0)
            del(df_jl)
            del(df_left_parent)
            # 
            df_right_parent=jrow.df_right
            df_grid=search_thresholds(df_p=df_right_parent.copy(), list_feature_cols=list_feature_cols, n_quantiles=n_quantiles)
            jfeat, feat_threshold, information_gain, df_left, df_right=get_nodes(df_p=df_right_parent.copy(), df_grid=df_grid.copy() )
            df_jr=pd.DataFrame({'node_id':[ str(node_level) + '_' + str(jdx) + 'r' ], 'node_level':[node_level], 'feature':[jfeat], 'threshold':[feat_threshold], 
                              'information_gain':[information_gain], 'df_left':[df_left], 'df_right':[df_right], 'parent_node_id':[jrow.node_id]})
            df_node_agg=pd.concat([df_node_agg, df_jr],axis=0)
            del(df_jr)
            del(df_right_parent)
        del(df_node_parent)
        df_node_agg=df_node_agg.reset_index(drop=True)
    else:
        print('Error! node_level=', node_level)
        raise ValueError
    df_summary=pd.concat([df_summary, df_node_agg],axis=0).reset_index(drop=True)
    del(df_node_agg)

node_level= 0
quantile=
    quantile   hour   wind
0      0.25   5.75  2.875
1      0.50  11.50  4.800
2      0.75  17.25  7.050
node_level= 1
**
left parent type= <class 'pandas.core.frame.DataFrame'>
quantile=
    quantile  hour   wind
0      0.25  2.75  2.675
1      0.50  5.50  4.800
2      0.75  8.25  6.600
quantile=
    quantile   hour   wind
0      0.25  14.75  2.900
1      0.50  17.50  4.900
2      0.75  20.25  8.325
node_level= 2
**
left parent type= <class 'pandas.core.frame.DataFrame'>
quantile=
    quantile  hour  wind
0      0.25   2.0   3.1
1      0.50   4.0   5.0
2      0.75   6.0   6.6
quantile=
    quantile  hour  wind
0      0.25   9.0   2.4
1      0.50  10.0   3.5
2      0.75  11.0   6.1
**
left parent type= <class 'pandas.core.frame.DataFrame'>
quantile=
    quantile  hour   wind
0      0.25  13.0  2.800
1      0.50  14.5  3.800
2      0.75  16.0  8.125
quantile=
    quantile  hour   wind
0      0.25  19.0  3.325
1      0.50  20.5  5.800
2      0.75  22.0  8.850


In [13]:
df_summary

,node_id,node_level,feature,threshold,information_gain,df_left,df_right,parent_node_id
0,0_0,0,hour,11.500,0.471813,datetime temperature high low ...,datetime temperature high low ...,NaN
1,1_0l,1,hour,8.250,0.483131,datetime temperature high low ...,datetime temperature high low ...,0_0
2,1_0r,1,hour,17.500,0.276642,datetime temperature high low ...,datetime temperature high low ...,0_0
3,2_0l,2,hour,4.000,0.054358,datetime temperature high low ...,datetime temperature high low ...,1_0l
4,2_0r,2,hour,10.000,0.156982,datetime temperature high low w...,datetime temperature high low ...,1_0l
5,2_1l,2,wind,8.125,0.119092,datetime temperature high low ...,datetime temperature high low ...,1_0r
6,2_1r,2,wind,5.800,0.239316,datetime temperature high low ...,datetime temperature high low ...,1_0r


In [9]:
get_nodes(df_p=df_w.copy(), df_grid=df_grid.copy())

['hour',
 9.0,
 0.6931166439341212,
               datetime  temperature high low  wind quolity direction  hour
 0  2022-01-14 01:00:00          3.9    8   1   7.2       8        北西   1.0
 1  2022-01-14 02:00:00          2.4    8   1   5.5       8        北西   2.0
 2  2022-01-14 03:00:00          2.2    8   1   7.5       8        北西   3.0
 3  2022-01-14 04:00:00          2.6    8   1   6.8       8       北北西   4.0
 4  2022-01-14 05:00:00          2.8    8   1   8.7       8       北北西   5.0
 ..                 ...          ...  ...  ..   ...     ...       ...   ...
 76 2022-01-22 05:00:00          3.1    8   1   6.6       8        北西   5.0
 77 2022-01-22 06:00:00          2.9    8   1   7.0       8       北北西   6.0
 78 2022-01-22 07:00:00          2.9    8   1   6.6       8        北西   7.0
 79 2022-01-22 08:00:00          3.7    8   1   5.4       8        北西   8.0
 80 2022-01-23 00:00:00          5.9    8   1   3.1       8       北北西   0.0
 
 [81 rows x 8 columns],
                datetime  

In [37]:
# df_w[:17]
df_w[df_w[jfeat]>feat_threshold][target_col]
del(ym)

In [44]:
df_w.dtypes
list_feature_cols


['hour', 'wind']

In [10]:
train_X=df_w.values
train_X

array([[Timestamp('2022-01-14 01:00:00'), 3.9, '8', ..., '8', '北西', 1.0],
       [Timestamp('2022-01-14 02:00:00'), 2.4, '8', ..., '8', '北西', 2.0],
       [Timestamp('2022-01-14 03:00:00'), 2.2, '8', ..., '8', '北西', 3.0],
       ...,
       [Timestamp('2022-01-22 22:00:00'), 6.9, '8', ..., '8', '北東', 22.0],
       [Timestamp('2022-01-22 23:00:00'), 6.8, '8', ..., '8', '北', 23.0],
       [Timestamp('2022-01-23 00:00:00'), 5.9, '8', ..., '8', '北北西', 0.0]],
      dtype=object)

In [35]:
#for jrow in train_X:
#    print(jrow[0], jrow[1], jrow[2], jrow[3])
for l0 in df_w.values:
    print(l0)

[Timestamp('2022-01-14 01:00:00') 3.9 '8' '1' 7.2 '8' '北西' 1.0]
[Timestamp('2022-01-14 02:00:00') 2.4 '8' '1' 5.5 '8' '北西' 2.0]
[Timestamp('2022-01-14 03:00:00') 2.2 '8' '1' 7.5 '8' '北西' 3.0]
[Timestamp('2022-01-14 04:00:00') 2.6 '8' '1' 6.8 '8' '北北西' 4.0]
[Timestamp('2022-01-14 05:00:00') 2.8 '8' '1' 8.7 '8' '北北西' 5.0]
[Timestamp('2022-01-14 06:00:00') 2.1 '8' '1' 5.3 '8' '北西' 6.0]
[Timestamp('2022-01-14 07:00:00') 1.6 '8' '1' 5.0 '8' '北西' 7.0]
[Timestamp('2022-01-14 08:00:00') 2.8 '8' '1' 4.8 '8' '北西' 8.0]
[Timestamp('2022-01-14 09:00:00') 4.2 '8' '1' 2.9 '8' '北西' 9.0]
[Timestamp('2022-01-14 10:00:00') 5.5 '8' '1' 3.5 '8' '西北西' 10.0]
[Timestamp('2022-01-14 11:00:00') 7.0 '8' '1' 4.3 '8' '北西' 11.0]
[Timestamp('2022-01-14 12:00:00') 7.6 '8' '1' 4.0 '8' '西北西' 12.0]
[Timestamp('2022-01-14 13:00:00') 8.8 '8' '1' 5.8 '8' '西' 13.0]
[Timestamp('2022-01-14 14:00:00') 8.6 '8' '1' 6.4 '8' '西北西' 14.0]
[Timestamp('2022-01-14 15:00:00') 8.6 '8' '1' 8.3 '8' '北西' 15.0]
[Timestamp('2022-01-14 16:00:0

In [23]:
df_w.dtypes

datetime       object
temperature    object
high           object
low            object
wind           object
quolity        object
direction      object
dtype: object